# Preliminary Analysis

BNPL Data timeline: 2021-2-28 to 2022-10-26

In [363]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.functions import* 
from pyspark.sql.functions import regexp_replace, col, trim, split
import pandas as pd
import geopandas as gpd

In [364]:
spark = (
    SparkSession.builder.appName("Preliminary Analysis")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.driver.memory","4G")
    .config("spark.executor.memory","4G")
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC")
    .getOrCreate()
)

# BNPL dataset

# Table 1

In [365]:
consumer_fraud = spark.read.csv('../data/tables/tables 1/consumer_fraud_probability.csv', header=True, inferSchema=True)
merchant_fraud = spark.read.csv('../data/tables/tables 1/merchant_fraud_probability.csv', header=True, inferSchema=True)
consumer_user_details = spark.read.parquet('../data/tables/tables 1/consumer_user_details.parquet')
tbl_consumer = spark.read.csv('../data/tables/tables 1/tbl_consumer.csv', header=True, inferSchema=True)
tbl_merchants = spark.read.parquet('../data/tables/tables 1/tbl_merchants.parquet')

In [366]:
# split tbl_consumer table
# single column into multiple columns
split_col = split(tbl_consumer['name|address|state|postcode|gender|consumer_id'], r'\|')

# create separate columns for each part
tbl_consumer = tbl_consumer.withColumn('name', split_col.getItem(0)) \
                           .withColumn('address', split_col.getItem(1)) \
                           .withColumn('state', split_col.getItem(2)) \
                           .withColumn('postcode', split_col.getItem(3)) \
                           .withColumn('gender', split_col.getItem(4)) \
                           .withColumn('consumer_id', split_col.getItem(5))

tbl_consumer = tbl_consumer.drop('name|address|state|postcode|gender|consumer_id')

tbl_consumer.show(3)

+----------------+--------------------+-----+--------+------+-----------+
|            name|             address|state|postcode|gender|consumer_id|
+----------------+--------------------+-----+--------+------+-----------+
|Yolanda Williams|413 Haney Gardens...|   WA|    6935|Female|    1195503|
|      Mary Smith|     3764 Amber Oval|  NSW|    2782|Female|     179208|
|   Jill Jones MD|  40693 Henry Greens|   NT|     862|Female|    1194530|
+----------------+--------------------+-----+--------+------+-----------+
only showing top 3 rows



# Join all comsumer, merchant table

In [367]:
consumer_table = consumer_fraud.join(consumer_user_details, on="user_id", how="inner")
consumer_table = consumer_table.join(tbl_consumer, on="consumer_id", how="inner")
consumer_table.show(10)

+-----------+-------+--------------+------------------+-----------------+--------------------+-----+--------+-----------+
|consumer_id|user_id|order_datetime| fraud_probability|             name|             address|state|postcode|     gender|
+-----------+-------+--------------+------------------+-----------------+--------------------+-----+--------+-----------+
|    1174371|     95|    2021-09-22|10.950213110987248|      Linda Burns|76786 Stephanie I...|  NSW|    2352|     Female|
|    1174371|     95|    2021-11-03| 9.077685805360991|      Linda Burns|76786 Stephanie I...|  NSW|    2352|     Female|
|     921339|    152|    2021-12-17|14.821132072309535|       Tina Clark|     637 Daniel View|  NSW|    2161|     Female|
|     612215|    275|    2021-06-04| 12.90435899477071|Dr. Nicholas Hill|36154 Rivera Neck...|  VIC|    3147|Undisclosed|
|    1493354|    378|    2021-10-24|15.080740281581361| William Sullivan|716 Samuel Rapids...|  VIC|    3533|       Male|
|    1493354|    378|   

In [368]:
consumer_table.select('postcode').distinct().count()

3161

In [369]:
merchant_table = merchant_fraud.join(tbl_merchants, on="merchant_abn", how="inner")
merchant_table.show(10)

+------------+--------------+------------------+--------------------+--------------------+
|merchant_abn|order_datetime| fraud_probability|                name|                tags|
+------------+--------------+------------------+--------------------+--------------------+
| 11149063370|    2022-02-25| 51.01538421455241|     Et Arcu Limited|([art dealers and...|
| 11149063370|    2021-11-14|52.407803322764764|     Et Arcu Limited|([art dealers and...|
| 11149063370|    2021-08-28| 56.43761254995139|     Et Arcu Limited|([art dealers and...|
| 11470993597|    2021-09-28| 63.37734364737917|      Sed Associates|((watch, clock, a...|
| 11590404675|    2021-12-21|29.607818240092094|         Arcu Sed PC|((antique shops -...|
| 14530561097|    2021-09-15| 80.80054474543395|        Duis At Inc.|[[jewelry, watch,...|
| 15043504837|    2021-10-08|25.054391991473924|   Odio Incorporated|([jewelry, watch,...|
| 15043504837|    2021-12-14| 26.12523097610844|   Odio Incorporated|([jewelry, watch,...|

# Concumer, merchant Preprocessing 

### Consumer table preprocessing

In [370]:
# check for invalid fraud probabilities - none
invalid_fraud_prob = consumer_table.filter((col("fraud_probability") < 0) | (col("fraud_probability") > 100))
invalid_fraud_prob.show(truncate=False)

+-----------+-------+--------------+-----------------+----+-------+-----+--------+------+
|consumer_id|user_id|order_datetime|fraud_probability|name|address|state|postcode|gender|
+-----------+-------+--------------+-----------------+----+-------+-----+--------+------+
+-----------+-------+--------------+-----------------+----+-------+-----+--------+------+



In [371]:
# check missing values
# all columns in the DataFrame
columns = consumer_table.columns

# count missing values for each column
missing_values_expr = [sum(col(c).isNull().cast("int")).alias(c) for c in columns]
missing_counts = consumer_table.select(*missing_values_expr).collect()[0].asDict()

# missing values count for each column
for column, count in missing_counts.items():
    print(f"There are {count} missing values in the '{column}' column.")

There are 0 missing values in the 'consumer_id' column.
There are 0 missing values in the 'user_id' column.
There are 0 missing values in the 'order_datetime' column.
There are 0 missing values in the 'fraud_probability' column.
There are 0 missing values in the 'name' column.
There are 0 missing values in the 'address' column.
There are 0 missing values in the 'state' column.
There are 0 missing values in the 'postcode' column.
There are 0 missing values in the 'gender' column.


In [372]:
# Save the DataFrame as a Parquet file
consumer_table.write.parquet('../data/curated/consumer', mode='overwrite')


### Merchant table preprocessing

In [373]:
# convert all string to lowercase
merchant_table = merchant_table.withColumn("name", lower(col("name"))) \
                               .withColumn("tags", lower(col("tags")))



In [374]:
# convert all brackets to []

# replace '(' with '['
merchant_table = merchant_table.withColumn("tags_converted", regexp_replace(col("tags"), r'\(', '['))

# replace ')' with ']'
merchant_table = merchant_table.withColumn("tags_converted", regexp_replace(col("tags_converted"), r'\)', ']'))



In [375]:
# split the elements by '], [' to get the three parts
split_col = split(col("tags_converted"), r'\], \[')

# clean up each part and assign them to separate columns
merchant_table = merchant_table.withColumn("category", trim(regexp_replace(split_col.getItem(0), r'^\[|\]$', ''))) \
                               .withColumn("revenue level", trim(regexp_replace(split_col.getItem(1), r'^\[|\]$', ''))) \
                               .withColumn("take_rate", trim(regexp_replace(split_col.getItem(2), r'^\[take rate: |\]$', '')))

# keep only numeric values
merchant_table = merchant_table.withColumn("category", regexp_replace(col("category"), r'^\[|\]$', ''))
merchant_table = merchant_table.withColumn("take_rate", regexp_replace(col("take_rate"), r'[^\d.]+', ''))

merchant_table.drop('tags', 'tags_converted')

merchant_abn,order_datetime,fraud_probability,name,category,revenue level,take_rate
11149063370,2022-02-25,51.01538421455241,et arcu limited,art dealers and g...,b,4.84
11149063370,2021-11-14,52.407803322764764,et arcu limited,art dealers and g...,b,4.84
11149063370,2021-08-28,56.43761254995139,et arcu limited,art dealers and g...,b,4.84
11470993597,2021-09-28,63.37734364737917,sed associates,"watch, clock, and...",d,1.35
11590404675,2021-12-21,29.607818240092094,arcu sed pc,antique shops - s...,b,4.19
14530561097,2021-09-15,80.80054474543395,duis at inc.,"jewelry, watch, c...",c,1.69
15043504837,2021-10-08,25.054391991473924,odio incorporated,"jewelry, watch, c...",b,4.62
15043504837,2021-12-14,26.12523097610844,odio incorporated,"jewelry, watch, c...",b,4.62
15043504837,2021-08-29,59.77648897297805,odio incorporated,"jewelry, watch, c...",b,4.62
15157368385,2021-12-13,64.2774131928303,tempus non lacini...,artist supply and...,b,3.98


In [376]:
# check for invalid fraud probabilities - none
invalid_fraud_prob = merchant_table.filter((col("fraud_probability") < 0) | (col("fraud_probability") > 100))
invalid_fraud_prob.show(truncate=False)


+------------+--------------+-----------------+----+----+--------------+--------+-------------+---------+
|merchant_abn|order_datetime|fraud_probability|name|tags|tags_converted|category|revenue level|take_rate|
+------------+--------------+-----------------+----+----+--------------+--------+-------------+---------+
+------------+--------------+-----------------+----+----+--------------+--------+-------------+---------+



In [377]:
# check missing values
# all columns in the DataFrame
columns = merchant_table.columns

# count missing values for each column
missing_values_expr = [sum(col(c).isNull().cast("int")).alias(c) for c in columns]
missing_counts = merchant_table.select(*missing_values_expr).collect()[0].asDict()

# missing values count for each column
for column, count in missing_counts.items():
    print(f"There are {count} missing values in the '{column}' column.")

There are 0 missing values in the 'merchant_abn' column.
There are 0 missing values in the 'order_datetime' column.
There are 0 missing values in the 'fraud_probability' column.
There are 0 missing values in the 'name' column.
There are 0 missing values in the 'tags' column.
There are 0 missing values in the 'tags_converted' column.
There are 0 missing values in the 'category' column.
There are 0 missing values in the 'revenue level' column.
There are 0 missing values in the 'take_rate' column.


In [378]:
# save the DataFrame as a Parquet file
merchant_table.write.parquet('../data/curated/merchant', mode='overwrite')


# Table 2 3 4 - transactions

In [379]:
# 3 transactions tables
tables_2 = spark.read.parquet('../data/tables/tables 2')
tables_3 = spark.read.parquet('../data/tables/tables 3')
tables_4 = spark.read.parquet('../data/tables/tables 4')

In [380]:
print('number of transactions in table 2 3 4: ', tables_2.count(), tables_3.count(), tables_4.count())


number of transactions in table 2 3 4:  3643266 4508106 6044133


In [381]:
# combine all transactions - 14195505 transactions with no duplicate record
transaction_table = tables_2.union(tables_3).union(tables_4)

In [382]:
transaction_table.show(5)

+-------+------------+------------------+--------------------+--------------+
|user_id|merchant_abn|      dollar_value|            order_id|order_datetime|
+-------+------------+------------------+--------------------+--------------+
|  18478| 62191208634|63.255848959735246|949a63c8-29f7-4ab...|    2021-08-20|
|      2| 15549624934| 130.3505283105634|6a84c3cf-612a-457...|    2021-08-20|
|  18479| 64403598239|120.15860593212783|b10dcc33-e53f-425...|    2021-08-20|
|      3| 60956456424| 136.6785200286976|0f09c5a5-784e-447...|    2021-08-20|
|  18479| 94493496784| 72.96316578355305|f6c78c1a-4600-4c5...|    2021-08-20|
+-------+------------+------------------+--------------------+--------------+
only showing top 5 rows



In [383]:
# check duplicate transaction records

# group by all columns and count occurrences
duplicates = transaction_table.groupBy(transaction_table.columns).count()

# keep only duplicate records
duplicates = duplicates.filter(col("count") > 1)

# duplicate row
duplicates.show()

+-------+------------+------------+--------+--------------+-----+
|user_id|merchant_abn|dollar_value|order_id|order_datetime|count|
+-------+------------+------------+--------+--------------+-----+
+-------+------------+------------+--------+--------------+-----+



In [384]:
# check missing values
# all columns in the DataFrame
columns = transaction_table.columns

# count missing values for each column
missing_values_expr = [sum(col(c).isNull().cast("int")).alias(c) for c in columns]
missing_counts = transaction_table.select(*missing_values_expr).collect()[0].asDict()

# missing values count for each column
for column, count in missing_counts.items():
    print(f"There are {count} missing values in the '{column}' column.")

There are 0 missing values in the 'user_id' column.
There are 0 missing values in the 'merchant_abn' column.
There are 0 missing values in the 'dollar_value' column.
There are 0 missing values in the 'order_id' column.
There are 0 missing values in the 'order_datetime' column.


In [385]:
# save the DataFrame as a Parquet file
transaction_table.write.parquet('../data/curated/transaction', mode='overwrite')


# External dataset

### Location datasets (Suburb and postcode)
1. https://www.abs.gov.au/statistics/standards/australian-statistical-geography-standard-asgs-edition-3/jul2021-jun2026/access-and-downloads/digital-boundary-files/SA2_2021_AUST_SHP_GDA2020.zip
2. https://data.gov.au/data/dataset/6cd8989d-4aca-46b7-b93e-77befcffa0b6/resource/cb659d81-5bd2-41f5-a3d0-67257c9a5893/download/asgs2021codingindexs.zip

In [386]:
# read SA2 name and its geometry
gda = gpd.read_file('../data/tables/SA2_2021_AUST_SHP_GDA2020/SA2_2021_AUST_GDA2020.shp')
gda = gda[['SA2_NAME21', 'geometry']]

# read the SA2 name and postcode lookup table 
postcode_sa2 = pd.read_csv('../data/tables/asgs2021codingindexs/2023 Locality to 2021 SA2 Coding Index.csv')
postcode_sa2 = postcode_sa2[['SA2_NAME_2021', 'POSTCODE']]
postcode_sa2 = postcode_sa2.dropna(subset=['POSTCODE'])
postcode_sa2['POSTCODE'] = postcode_sa2['POSTCODE'].astype(int)
postcode_sa2 = postcode_sa2.rename(columns={'POSTCODE': 'postcode'})

In [387]:
gda.head()

,SA2_NAME21,geometry
0,Braidwood,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4..."
1,Karabar,"POLYGON ((149.21899 -35.36738, 149.218 -35.366..."
2,Queanbeyan,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3..."
3,Queanbeyan - East,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3..."
4,Queanbeyan West - Jerrabomberra,"POLYGON ((149.19572 -35.36126, 149.1997 -35.35..."


In [388]:
postcode_sa2.head()

,SA2_NAME_2021,postcode
0,Black Mountain,2600
1,Parkes (ACT) - South,2600
2,Barton,2600
3,Deakin,2600
4,Parkes (ACT) - South,2600


In [389]:
postcode_sa2_geo = gda.merge(postcode_sa2, left_on='SA2_NAME21', right_on='SA2_NAME_2021', how='inner')
postcode_sa2_geo = postcode_sa2_geo.drop_duplicates()
postcode_sa2_geo = postcode_sa2_geo.drop(columns=['SA2_NAME_2021'])
postcode_sa2_geo = postcode_sa2_geo.rename(columns={
    'SA2_NAME21': 'SA2 name'
})
postcode_sa2_geo.head()

,SA2 name,geometry,postcode
0,Braidwood,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4...",2580
5,Braidwood,"POLYGON ((149.58424 -35.44426, 149.58444 -35.4...",2622
53,Karabar,"POLYGON ((149.21899 -35.36738, 149.218 -35.366...",2620
56,Queanbeyan,"POLYGON ((149.21326 -35.34325, 149.21619 -35.3...",2620
60,Queanbeyan - East,"POLYGON ((149.24034 -35.34781, 149.24024 -35.3...",2620


In [390]:
# number of unique postcode
unique_postcodes_count = len(postcode_sa2_geo['postcode'].unique())
print(f"Number of unique postcodes: {unique_postcodes_count}")

Number of unique postcodes: 2643


In [391]:
postcode_sa2_geo.shape

(4831, 3)

In [392]:
# Save the DataFrame as a CSV file
postcode_sa2_geo.to_file('../data/curated/postcode_sa2_geo.shp')


### Income dataset
https://www.abs.gov.au/statistics/labour/earnings-and-working-conditions/personal-income-australia/2020-21-financial-year/Table%201%20-%20Total%20income%2C%20earners%20and%20summary%20statistics%20by%20geography%2C%202016-17%20to%202020-21.xlsx


In [393]:
# total income dataset
# read the fourth Excel sheet
income_pandas = pd.read_excel('../data/tables/Table 1 - Total income, earners and summary statistics by geography, 2016-17 to 2020-21.xlsx', sheet_name=4, header=[5, 6])

In [394]:
# column of interest
columns_of_interest = [
    ('Unnamed: 1_level_0', 'SA2 NAME'),
    ('Earners (persons)', '2020-21'),
    ('Sum ($)',  '2020-21')
]
income_pandas = income_pandas.loc[:, columns_of_interest]

# rename the columns
income_pandas.columns = ['SA2 name', 'earners', 'income']

# remove the specific row with NaNs
income_pandas = income_pandas[income_pandas.notna().all(axis=1)]

# reset the index to maintain a clean index
income_pandas = income_pandas.reset_index(drop=True)

# convert Pandas DataFrame to Spark DataFrame
income_pyspark = spark.createDataFrame(income_pandas)
income_pyspark.show()

+--------------------+-------+-----------+
|            SA2 name|earners|     income|
+--------------------+-------+-----------+
|           Braidwood|  2,467|169,986,703|
|             Karabar|  5,103|355,538,349|
|          Queanbeyan|  7,028|486,157,371|
|   Queanbeyan - East|  3,398|252,003,459|
|Queanbeyan West -...|  8,422|774,662,009|
|             Googong|  3,555|331,654,182|
|Queanbeyan Surrounds| 10,647|923,249,146|
|             Bombala|  1,399| 80,343,596|
|               Cooma|  3,752|227,153,809|
|     Cooma Surrounds|  2,045|122,806,651|
|Jindabyne - Berri...|  4,652|287,143,356|
|        Batemans Bay|  4,314|204,396,825|
|Batemans Bay - South|  4,773|256,006,252|
|       Bega - Tathra|  5,086|282,411,339|
|Bega-Eden Hinterland|  5,499|258,027,869|
|   Broulee - Tomakin|  2,200|132,283,166|
|   Deua - Wadbilliga|     15|    881,592|
|                Eden|  1,792| 89,113,914|
|Eurobodalla Hinte...|  2,029|103,578,656|
|Merimbula - Tura ...|  6,694|365,742,677|
+----------

In [395]:
# save the DataFrame as a Parquet file
income_pyspark.write.parquet('../data/curated/income', mode='overwrite')


### Population dataset
https://www.abs.gov.au/statistics/people/population/regional-population/2021-22/32180DS0003_2001-22r.xlsx

In [396]:
# read the first Excel sheet
population_pandas = pd.read_excel('../data/tables/32180DS0003_2001-22r.xlsx', sheet_name=1, header=[6, 7])
population_pandas.head()

,Unnamed: 0_level_0,Unnamed: 1_level_0,Unnamed: 2_level_0,Unnamed: 3_level_0,Unnamed: 4_level_0,Unnamed: 5_level_0,Unnamed: 6_level_0,Unnamed: 7_level_0,Unnamed: 8_level_0,Unnamed: 9_level_0,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
,S/T code,S/T name,GCCSA code,GCCSA name,SA4 code,SA4 name,SA3 code,SA3 name,SA2 code,SA2 name,...,no.,no.,no.,no.,no.,no.,no.,no.,no.,no.
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,New South Wales,12.0,Rest of NSW,101.0,Capital Region,10102.0,Queanbeyan,101021007.0,Braidwood,...,3685,3762,3849,3950.0,4041.0,4145.0,4218.0,4282.0,4332.0,4366.0
2,1,New South Wales,12.0,Rest of NSW,101.0,Capital Region,10102.0,Queanbeyan,101021008.0,Karabar,...,8848,8731,8603,8531.0,8530.0,8516.0,8500.0,8535.0,8548.0,8528.0
3,1,New South Wales,12.0,Rest of NSW,101.0,Capital Region,10102.0,Queanbeyan,101021009.0,Queanbeyan,...,11050,11199,11213,11230.0,11362.0,11460.0,11468.0,11460.0,11375.0,11390.0
4,1,New South Wales,12.0,Rest of NSW,101.0,Capital Region,10102.0,Queanbeyan,101021010.0,Queanbeyan - East,...,4983,4967,4961,4970.0,5016.0,5079.0,5126.0,5089.0,5097.0,5090.0


In [397]:
print(population_pandas.columns)

MultiIndex([('Unnamed: 0_level_0',   'S/T code'),
            ('Unnamed: 1_level_0',   'S/T name'),
            ('Unnamed: 2_level_0', 'GCCSA code'),
            ('Unnamed: 3_level_0', 'GCCSA name'),
            ('Unnamed: 4_level_0',   'SA4 code'),
            ('Unnamed: 5_level_0',   'SA4 name'),
            ('Unnamed: 6_level_0',   'SA3 code'),
            ('Unnamed: 7_level_0',   'SA3 name'),
            ('Unnamed: 8_level_0',   'SA2 code'),
            ('Unnamed: 9_level_0',   'SA2 name'),
            (                2001,        'no.'),
            (                2002,        'no.'),
            (                2003,        'no.'),
            (                2004,        'no.'),
            (                2005,        'no.'),
            (                2006,        'no.'),
            (                2007,        'no.'),
            (                2008,        'no.'),
            (                2009,        'no.'),
            (                2010,        'no.'),


In [398]:
# columns of interest
columns_of_interest = [
    ('Unnamed: 9_level_0', 'SA2 name'),
    (2021, 'no.'),
    (2022, 'no.')
]

# selected columns
population_selected = population_pandas.loc[:, columns_of_interest]

# rename the columns
population_selected.columns = ['SA2 name', '2021', '2022']

# selected columns before removing the row
print("Before removing the NaN row:")
print(population_selected.head())

# remove the specific row with NaNs
population_selected_cleaned = population_selected[population_selected.notna().all(axis=1)]

# reset the index to maintain a clean index
population_selected_cleaned = population_selected_cleaned.reset_index(drop=True)

print("After removing the NaN row:")
print(population_selected_cleaned.head())

Before removing the NaN row:
            SA2 name     2021     2022
0                NaN      NaN      NaN
1          Braidwood   4332.0   4366.0
2            Karabar   8548.0   8528.0
3         Queanbeyan  11375.0  11390.0
4  Queanbeyan - East   5097.0   5090.0
After removing the NaN row:
                          SA2 name     2021     2022
0                        Braidwood   4332.0   4366.0
1                          Karabar   8548.0   8528.0
2                       Queanbeyan  11375.0  11390.0
3                Queanbeyan - East   5097.0   5090.0
4  Queanbeyan West - Jerrabomberra  12748.0  12779.0


In [399]:
population_selected_cleaned

,SA2 name,2021,2022
0,Braidwood,4332.0,4366.0
1,Karabar,8548.0,8528.0
2,Queanbeyan,11375.0,11390.0
3,Queanbeyan - East,5097.0,5090.0
4,Queanbeyan West - Jerrabomberra,12748.0,12779.0
...,...,...,...
2450,Christmas Island,1717.0,1787.0
2451,Cocos (Keeling) Islands,603.0,614.0
2452,Jervis Bay,309.0,311.0
2453,Norfolk Island,2221.0,2213.0


In [400]:
# Check for remaining NaN values
missing_value = population_selected_cleaned.isna().sum()

# Display the summary of NaN values
print("NaN values summary:")
print(missing_value)

NaN values summary:
SA2 name    0
2021        0
2022        0
dtype: int64


In [401]:
import pandas as pd

# check if '2021' and '2022' columns are present
if '2021' in population_selected_cleaned.columns and '2022' in population_selected_cleaned.columns:
    
    # create a new column 'average_population' that averages the '2021' and '2022' columns
    population_selected_cleaned['average_population'] = population_selected_cleaned[['2021', '2022']].mean(axis=1)
    
    # drop the '2021' and '2022' columns
    population_selected_cleaned = population_selected_cleaned.drop(columns=['2021', '2022'])
    
else:
    print("The columns '2021' and '2022' are not present in the DataFrame.")

# dataFrame with the average column
print("DataFrame after averaging population of 2021 and 2022:")
print(population_selected_cleaned)



DataFrame after averaging population of 2021 and 2022:
                             SA2 name  average_population
0                           Braidwood              4349.0
1                             Karabar              8538.0
2                          Queanbeyan             11382.5
3                   Queanbeyan - East              5093.5
4     Queanbeyan West - Jerrabomberra             12763.5
...                               ...                 ...
2450                 Christmas Island              1752.0
2451          Cocos (Keeling) Islands               608.5
2452                       Jervis Bay               310.0
2453                   Norfolk Island              2217.0
2454                  TOTAL AUSTRALIA          25845476.0

[2455 rows x 2 columns]


In [402]:
# convert Pandas DataFrame to Spark DataFrame
population = spark.createDataFrame(population_selected_cleaned)
population.show(5)

+--------------------+------------------+
|            SA2 name|average_population|
+--------------------+------------------+
|           Braidwood|            4349.0|
|             Karabar|            8538.0|
|          Queanbeyan|           11382.5|
|   Queanbeyan - East|            5093.5|
|Queanbeyan West -...|           12763.5|
+--------------------+------------------+
only showing top 5 rows



In [403]:
# save as a Parquet file
population.write.parquet('../data/curated/population', mode='overwrite')


### Unemployment dataset
https://explore.data.abs.gov.au/vis?tm=unemployment%20sa2&pg=0&df[ds]=C21_ASGS&df[id]=C21_G43_SA2&df[ag]=ABS&df[vs]=1.0.0&pd=2021%2C&dq=LFS_P1.3...&ly[rs]=REGION

In [404]:
# read the CSV file
unemployment = pd.read_csv('../data/tables/ABS_C21_G43_SA2_1.0.0_LFS_P1.3....csv')
unemployment.head(5)

,DATAFLOW,LFEMP: Selected Labour Force/Education/Migration characteristic,SEXP: Sex,REGION: Region,REGION_TYPE: Region Type,STATE: State,TIME_PERIOD: Time Period,OBS_VALUE
0,ABS:C21_G43_SA2(1.0.0),LFS_P1: Labour force status: % Unemployment,3: Persons,101021611: Queanbeyan Surrounds,SA2: Statistical Area Level 2,1: New South Wales,2021,2.6
1,ABS:C21_G43_SA2(1.0.0),LFS_P1: Labour force status: % Unemployment,3: Persons,107011134: Unanderra - Mount Kembla,SA2: Statistical Area Level 2,1: New South Wales,2021,4.2
2,ABS:C21_G43_SA2(1.0.0),LFS_P1: Labour force status: % Unemployment,3: Persons,204031069: Bright - Mount Beauty,SA2: Statistical Area Level 2,2: Victoria,2021,1.8
3,ABS:C21_G43_SA2(1.0.0),LFS_P1: Labour force status: % Unemployment,3: Persons,211051280: Montrose,SA2: Statistical Area Level 2,2: Victoria,2021,3.1
4,ABS:C21_G43_SA2(1.0.0),LFS_P1: Labour force status: % Unemployment,3: Persons,213051468: Werribee - West,SA2: Statistical Area Level 2,2: Victoria,2021,6.2


In [405]:
# select the columns 'REGION' and 'obs_value'
selected_columns = unemployment[['REGION: Region', 'OBS_VALUE']]

# select and rename the columns
selected_columns = unemployment[['REGION: Region', 'OBS_VALUE']]
selected_columns.columns = ['SA2 name', 'unemployment_rate']

# selected columns
print(selected_columns.head(5))

                              SA2 name  unemployment_rate
0      101021611: Queanbeyan Surrounds                2.6
1  107011134: Unanderra - Mount Kembla                4.2
2     204031069: Bright - Mount Beauty                1.8
3                  211051280: Montrose                3.1
4           213051468: Werribee - West                6.2


In [406]:
# remove numbers and colon from the 'SA2 name' column
selected_columns.loc[:, 'SA2 name'] = selected_columns['SA2 name'].str.replace(r'^\d+:\s*', '', regex=True)

# display the DataFrame with the cleaned 'SA2 name'
print(selected_columns.head(5))

                   SA2 name  unemployment_rate
0      Queanbeyan Surrounds                2.6
1  Unanderra - Mount Kembla                4.2
2     Bright - Mount Beauty                1.8
3                  Montrose                3.1
4           Werribee - West                6.2


/var/folders/hx/0_8y1ngj2zx7c7x2zn44jf1r0000gn/T/ipykernel_5820/2321517987.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_columns.loc[:, 'SA2 name'] = selected_columns['SA2 name'].str.replace(r'^\d+:\s*', '', regex=True)


In [407]:
# convert Pandas DataFrame to Spark DataFrame
unemployment = spark.createDataFrame(selected_columns)
unemployment.show(5)

+--------------------+-----------------+
|            SA2 name|unemployment_rate|
+--------------------+-----------------+
|Queanbeyan Surrounds|              2.6|
|Unanderra - Mount...|              4.2|
|Bright - Mount Be...|              1.8|
|            Montrose|              3.1|
|     Werribee - West|              6.2|
+--------------------+-----------------+
only showing top 5 rows



In [408]:
# save the DataFrame as a Parquet file
unemployment.write.parquet('../data/curated/unemployment', mode='overwrite')


# Merge all external datasets

In [409]:
# merge income and population
df_income_pop = population.join(income_pyspark, on='SA2 name', how='inner')

# merge the above with unemployment rate
df_income_pop_unemp = df_income_pop.join(unemployment, on='SA2 name', how='inner')

In [410]:
df_income_pop_unemp_pandas = df_income_pop_unemp.toPandas()

In [411]:
df_income_pop_unemp_post = pd.merge(df_income_pop_unemp_pandas, postcode_sa2_geo, on='SA2 name', how='inner')

In [412]:
df_income_pop_unemp_post.head()

,SA2 name,average_population,earners,income,unemployment_rate,geometry,postcode
0,Kiama Downs - Minnamurra,6002.0,"3,633","255,198,601",3.2,"POLYGON ((150.84893 -34.63408, 150.84936 -34.6...",2533
1,Raymond Terrace,14802.0,"7,118","390,229,618",7.2,"POLYGON ((151.74096 -32.75945, 151.74258 -32.7...",2322
2,Raymond Terrace,14802.0,"7,118","390,229,618",7.2,"POLYGON ((151.74096 -32.75945, 151.74258 -32.7...",2324
3,Niagara Park - Lisarow,8215.5,"4,877","315,596,744",3.9,"POLYGON ((151.34439 -33.38197, 151.34385 -33.3...",2250
4,Wellington,8932.5,"3,993","201,898,931",6.0,"POLYGON ((148.7241 -32.49449, 148.72418 -32.49...",2818
